In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import json
import requests
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
# 구별로 json 불러오기

url = 'https://www.daegufood.go.kr/kor/api/tasty.html?mode=json&addr='
url_jung = url + '%EC%A4%91%EA%B5%AC'
url_dong = url + '%EB%8F%99%EA%B5%AC'
url_seo = url + '%EC%84%9C%EA%B5%AC'
url_nam = url + '%EB%82%A8%EA%B5%AC'
url_buk = url + '%EB%B6%81%EA%B5%AC'
url_su = url + '%EC%88%98%EC%84%B1%EA%B5%AC'
url_dal = url + '%EB%8B%AC%EC%84%9C%EA%B5%AC'
url_daln = url + '%EB%8B%AC%EC%84%B1%EA%B5%B0'

r_jung = requests.get(url_jung)
r_dong = requests.get(url_dong)
r_seo = requests.get(url_seo)
r_nam = requests.get(url_nam)
r_buk = requests.get(url_buk)
r_su = requests.get(url_su)
r_dal = requests.get(url_dal)
r_daln = requests.get(url_daln)

json_jung = r_jung.json(strict=False)
json_dong = r_dong.json(strict=False)
json_seo = r_seo.json(strict=False)
json_nam = r_nam.json(strict=False)
json_buk = r_buk.json(strict=False)
json_su = r_su.json(strict=False)
json_dal = r_dal.json(strict=False)
json_daln = r_daln.json(strict=False)

dg_mat = [json_jung, json_dong, json_seo, json_nam, json_buk, json_su, json_dal, json_daln]

200
200
200
200
200
200
200
200


In [23]:
dg_list=[]
for i in range (len(dg_mat)) :
    for j in range (len(dg_mat[i]['data'])) :
        dg_list.append((dg_mat[i]['data'][j]['BZ_NM'], dg_mat[i]['data'][j]['GNG_CS']))

adr = pd.DataFrame(dg_list, columns=['name', 'address'])

adr.head()

,name,address
0,송림식당,대구광역시 중구 삼덕동2가 219-1
1,빈플러스 테이스팅 스푼(TASTING SPOON),대구광역시 중구 삼덕동1가 21-10
2,박석규 중앙떡볶이,대구광역시 중구 동성로2가 70-1
3,가스트로락,대구광역시 중구 대봉동 31-67
4,차이,대구광역시 중구 삼덕동1가 14-3


In [65]:
adr.loc[900]

name                       가창우록리먹거리마을
address    대구광역시 달성군 가창면 우록리 / 삼산리 일대
Name: 900, dtype: object

In [28]:
status_success = 0
adr_json = []

for i in tqdm_notebook(range(len(adr))) :   
    k = adr['address'][i]
    api = f'https://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address={k}&refine=true&simple=false&format=json&type=parcel&key=52C3080F-5F83-335E-B72D-6E5156073F68'
    req = requests.get(api)

    status_success+=1 if req.status_code==200 else print(f'{i}번째에서 에러남', status_code)
    if

    adr_json.append(req.json())
       
print('925 나오면 req 전부 양호 :', status_success)
        
        

925 나오면 req 전부 양호 : 925


In [45]:
adr_json[0]

{'response': {'service': {'name': 'address',
   'version': '2.0',
   'operation': 'getcoord',
   'time': '166(ms)'},
  'status': 'OK',
  'input': {'type': 'parcel', 'address': '대구광역시 중구 삼덕동2가 219-1'},
  'refined': {'text': '대구광역시 중구 삼덕동2가 219-1',
   'structure': {'level0': '대한민국',
    'level1': '대구광역시',
    'level2': '중구',
    'level3': '',
    'level4L': '삼덕동2가',
    'level4LC': '2711010600',
    'level4A': '삼덕동',
    'level4AC': '2711054500',
    'level5': '219-1',
    'detail': ''}},
  'result': {'crs': 'EPSG:4326',
   'point': {'x': '128.603541305001', 'y': '35.864931923605'}}}}

In [61]:
adr_json[900]

{'response': {'service': {'name': 'address',
   'version': '2.0',
   'operation': 'getcoord',
   'time': '115(ms)'},
  'status': 'NOT_FOUND',
  'record': {'total': '0', 'current': '0'},
  'page': {'total': '1', 'current': '1', 'size': '10'}}}

In [59]:
adr_json[900]['response']['refined']['structure']['level2']

KeyError: 'refined'

In [53]:
adr_list=[]
for i in range (len(adr_json)) :
    adr_list.append((adr_json[i]['response']['refined']['structure']['level2'],
                     adr_json[i]['response']['result']['point']['x'],
                     adr_json[i]['response']['result']['point']['y']))

adr = pd.DataFrame(adr_list, columns=['gu', 'x_longitude', 'y_latitude'])

adr.head()


KeyError: 'refined'